# Pandas

In [1]:
import pandas as pd

In [41]:
persons = pd.DataFrame({
    'firstname': ['Joerg', 'Johanna', 'Caro', 'Philipp'],
    'lastname': ['Faschingbauer', 'Faschingbauer', 'Faschingbauer', 'Lichtenberger'],
    'email': ['JF@faschingbauer.co.at', 'Caro@email.com', 'Johanna@email.com', 'PHILIPP@email.com'],
    'age': [56, 27, 25, 37],
})

#### Apply On Entire DataFrame

Default direction: 'rows' -> length of each column

In [ ]:
df.apply(len)

In [ ]:
df.apply(len, axis='columns')

#### ``applymap``: Each Element of DataFrama

### Add Rows: Assign To New Index

In [ ]:
df2 = df.copy()

Assign to one-past-last row with ``loc[]`` (doing the same with ``iloc[]`` would appear more logical, but it refuses to "enlarge the object")

In [ ]:
df2.loc[len(df2)] = ('Isolde', 'Haubentaucher', '666608091972', 50)

In [ ]:
df2

### Add Rows: ``append()``

Would be most expressive, but is deprecated

### Add Rows: Concatenating Frames

This appears to be the most recommended approach

In [ ]:
df.columns

In [ ]:
new_users = pd.DataFrame({'firstname': ['Isolde', 'Saskia'], 'lastname': ['Haubentaucher', 'Haubentaucher'], 'svnr': ['666608091972', '456710042003'], 'age': [50, 19]})

In [ ]:
df_whole = pd.concat([df, new_users])

In [ ]:
df_whole

### Groups, Aggregation

#### Naive Approach: Separate Filter for each possible value

In [ ]:
flt = df['lastname'] == 'Faschingbauer'
df.loc[flt, 'age'].mean()

In [ ]:
flt = df['lastname'] == 'Lichtenberger'
df.loc[flt, 'age'].mean()

#### ``groupby``

In [ ]:
lastnames = df.groupby('lastname')

In [ ]:
lastnames

**To give an idea what it is ...**

Converted to a list ...

In [ ]:
list(lastnames)

That into a ``dict`` ... looks like the elements are data frames

In [ ]:
d = dict(list(lastnames))

In [ ]:
d

In [ ]:
type(d['Faschingbauer'])

#### Working with ``groupby``

``get_group()`` gives a DataFrame

In [ ]:
fasch = lastnames.get_group('Faschingbauer')

In [ ]:
fasch

Popular use: aggregation on each group

In [ ]:
fasch['age'].mean()

In [ ]:
fasch['age'].sum()

In [ ]:
try:
    lastnames.get_group('Notexist')
except KeyError as e:
    print(type(e), e)